In [ ]:
#import socket
import time
import cv2
import numpy as np
from realsense_depth import *
from IPython.display import clear_output

buffer = [0] * 40  #buffer moving average filter
dc = DepthCamera() #initialize intel realsense
position = 0       #degrees

#set frames (if no red object detected)
ret, depth_frame, color_frame, depth_frame2 = dc.get_frame()
rows, cols, _ = color_frame.shape
x_medium = int(cols/2)
y_medium = int(rows/2)
center = int(cols/2)

#checkPoints



while True:
    
    #stream from intel realsense
    ret, depth_frame, color_frame, depth_frame2 = dc.get_frame()
    kernel = np.ones((2,2), np.uint8)
    depth_frame2 = cv2.dilate(depth_frame2, kernel, iterations=25)
    depth_frame = cv2.dilate(depth_frame, kernel, iterations=25)
    depth_frame = cv2.blur(depth_frame,(2,2)) 
    depth_frameDISP = cv2.normalize(depth_frame, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    #get mask for all pixels within distance threshold
    distMask = np.zeros(depth_frameDISP.shape)
    locs = np.where(depth_frame < 300, depth_frame, 0)
    distMask = distMask+locs
    distMask = cv2.dilate(distMask, kernel, iterations=20)
    distMask = cv2.erode(distMask, kernel, iterations=30)
    distMask = distMask.astype(np.uint8)
    
    distMask2 = np.zeros(depth_frameDISP.shape)
    locs2 = np.where((depth_frame > 300) & (depth_frame < 700), depth_frame, 0)
    distMask2 = distMask2+locs2
    distMask2 = cv2.dilate(distMask2, kernel, iterations=20)
    distMask2 = cv2.erode(distMask2, kernel, iterations=30)
    distMask2 = distMask2.astype(np.uint8)
    
    
    #finding largest object
    contours, _ = cv2.findContours(distMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x:cv2.contourArea(x), reverse=True)

    #this follows the largest object within the distance
    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)
        #if w*h < 1000:
            #continue
        #else:
        cv2.rectangle(color_frame, (x,y), (x + w, y + h), (0, 255, 0), 2)
        x_medium = int((x + x + w)/2)
        y_medium = int((y + y + h)/2)
        cv2.circle(color_frame, (x_medium,y_medium), 5, (0, 255, 0))
        break
    
    #use moving average buffer to stabilize output distance
    dist1 = depth_frame[y_medium, x_medium]
    buffer[0] = dist1
    for i in range(len(buffer)):
        buffer[len(buffer)-1-i] = buffer[len(buffer)-2-i]
    dist1 = round(np.mean(dist1)/100)*100
    
    clear_output(wait=True)
    print(buffer)
    print(dist1)
    print(position)
    
    #shows frame
    cv2.imshow('mask', distMask)
    cv2.imshow('depth', depth_frameDISP)
    cv2.imshow('color_frame', color_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

    #movement commands
    if x_medium < center - 100:
        position += 1
    elif x_medium > center + 100:
        position -= 1
    if(dist1 < 800):
        print('stop')

cv2.destroyAllWindows